<a href="https://colab.research.google.com/github/zzwony/Start_0920/blob/main/01_06_tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# gdrive에 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
# 의존성 패키지 설치
!pip install ratsnlp

## GPT 입력값 만들기

GPT 모델 입력값을 만들려면 Byte-level Byte Pair Encoding 어휘집합 구축 결과(`vocab.json`, `merges.txt`)가 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 있어야 합니다. 다음을 수행해 이미 만들어 놓은 BBPE 어휘집합을 포함한 GPT 토크나이저를 `tokenizer_gpt`라는 변수로 선언합니다.


In [ ]:
# GPT 선언
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained('/gdrive/MyDrive/nlpbook/bbpe')
tokenizer_gpt.pad_token = '[PAD]'

file /gdrive/MyDrive/nlpbook/bbpe/config.json not found


예시 문장 세 개를 각각 토큰화해보겠다.

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

토큰화 결과를 확인한다.

In [ ]:
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

이전 배치의 크기가 3이라고 가정하고 이번 배치의 입력값을 만들어 보겠다.

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_gpt(
    sentences,
    padding='max_length',
    max_length=12,
    truncation=True,
)

이 코드의 실행결과로 1) input_ids 2) attention_mask가 생성되었다.

batch_inputs의 내용을 확인해보겠다.

In [ ]:
batch_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
batch_inputs['input_ids']

[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292],
 [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

input_ids는 토큰화 결과를 가지고 각 토큰을 인덱스로 바꾼것이다. → 인덱싱 과정

(vocab.json 기준)

In [ ]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

attention_mask는 일반 토큰(1)과 패딩 토근(0)을 구분해 주는 장치이다.

# BERT

## BERT 입력값 만들기

In [ ]:
# BERT 토크나이저 선언
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(
    '/gdrive/MyDrive/nlpbook/wordpiece',
    do_lower_case=False,
)

file /gdrive/MyDrive/nlpbook/wordpiece/config.json not found


In [ ]:
# BERT 토크나이저로 토큰화하기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

예시 문장 3개를 워드피스 토크나이저로 토큰화한다.

(##은 해당 토큰 어절의 시작이 아니라 앞선 토큰과 같은 위치에 있으며 연속됨을 표시함.)

In [ ]:
tokenized_sentences

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터',
  '##보고',
  '초딩',
  '##영화',
  '##줄',
  '.',
  '.',
  '.',
  '.',
  '오버',
  '##연기',
  '##조차',
  '가볍',
  '##지',
  '않',
  '##구나'],
 ['별루', '였다', '.', '.']]

In [ ]:
# BERT 모델 입력 만들기
batch_inputs = tokenizer_bert(
    sentences,
    padding='max_length',
    max_length=12,
    truncation=True,
)

이 코드의 실행 결과로 1) input_ids 2) attention_mask 3) token_type_ids 가 만들어진다.

In [ ]:
batch_inputs['input_ids']

[[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1177, 16, 3],
 [2, 3274, 9508, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

출력물을 보면 맨 앞에는 2, 맨 뒤에는 3이 붙었다.

이는 [CLS], [SEP] 라는 토큰에 대응하는 인덱스이며,

BERT는 문장 시작과 끝에 이 2개의 토큰을 덧붙이는 특징이 있다.

In [ ]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

GPT와 마찬가지로 일반 토큰이 자리한 곳은 1, 패딩 토큰이 자리한 곳은 0을 구분해준다.

In [ ]:
batch_inputs['token_type_ids']

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

문장이 하나만 들어가서 감성 분석을 할때는 0 이라는 신호를 주어서 하나의 문장만 들어가게 하고,

0과 1이라는 신호를 주면 2개의 문장이 들어가서 추리&분석을 하라는 뜻이다.

위의 출력 결과가 전부 0이 나온 이유는 이번 실습에서 문장을 하나씩 넣었기 때문이다.

# 언어 모델

- 언어 모델: 단어 시퀀스에 확률을 부여하는 모델
---
- 결합 확률: '난폭' 이라는 단어와 '운전' 이라는 단어가 동시에 나타날 결합 확률을 말함.
 - P(무모, 운전) 보다는 P(난폭, 운전)이 큰 확률값을 지닌다.
---
- 조건부 확률: 결과가 되는 사건(운전)은 조건이 되는 사건(난폭)의 영향을 받아 변한다는 개념을 내포
 - 난폭이 나타난 다음에 운전이 나타날 확률
 - 언어 모델을 이전 단어들이 주어졌을 때 다음 단어가 나타날 확률을 부여하는 모델이라고 정의하기도 한다.
---
- 순방향 언어 모델: 문장 앞부터 뒤로, 사람이 이해하는 순서대로 계산하는 모델을 말함.
 - GPT, ELMo 같은 모델이 이런 방식으로 프리트레인을 수행함.
---
- 역방향 언어 모델: 문장 뒤부터 앞으로 계산하는 역방향 언어 모델
 - ELMo 같은 모델이 이런 방식으로 프리트레인을 수행함.
---
- P( w | context ) ; 넓은 의미의 언어 모델
 - 이는 컨텍스트(contect; 주변 맥락 정보)가 전제된 상태에서 특정 단어(w)가 나타날 조건부 확률을 나타냄.
 - 이렇게 정의된 언어 모델은 단어나 시퀀스로 구성된 컨텍스트를 입력받아 특정 단어가 나타날 확률을 출력함.
 - 이때 컨텍스트와 맞힐 단어를 어떻게 설정하는냐에 따라 다양하게 변형할 수 있다.
   - 1) 마스크 언어 모델
   - 2) 스킵-그램 모델
---
- 마스크 언어 모델
 - 학습 대상 문장에 빈칸을 만들어 놓고 해당 빈칸에 올 단어로 적절한 단어가 무엇일지 분류하는 과정으로 학습함.
 - BERT가 마스크 언어 모델로 프리트레인하는 대표적인 모델이다.
   - [MASK] 카페 갔었어 거기 사람 많더라.
   - 어제 [MASK] 갔었어 거기 사람 많더라.
   - 어제 카페 [MASK] 거기 사람 많더라.
   - 어제 카페 갔었어 [MASK] 사람 많더라.
   - 어제 카페 갔었어 거기 [MASK] 많더라.
   - 어제 카페 갔었어 거기 사람 [MASK].
     - 여기서 [MASK] 부분은 맞혀야 할 타깃 단어를 가리킨다.

 - [장점] 맞힐 단어 이전 단어들만 참고할 수 있는 순방향, 역방향 언어 모델과 달리 마스크 언어 모델은 맞힐 단어를 계산할 때 문장 전체의 맥락을 참고함.
  - 마스크 언어 모델에 양방향 성질이 있다고 함.
---
- 스캅-그램 모델
 - 어떤 단어 앞뒤에 특정 범위를 정해 두고 이 범위 내에 어떤 단어가 들어올지 분류한느 과정으로 학습함.
 - 컨텍스트로 설정한 단어(가려진 단어)의 앞뒤로 두 개씩 보는 상황이다.
   - 어제 카페 [가려진 단어] 거기 사람
   - 카페 갔었어 [가려진 단어] 사람 많더라
     - 이때 스킵-그램 모델은 [갔었어] 주변에 어제, 카페, 거기, 사람이 나타날 확률을 각각 높이는 방식으로 학습함.
     - 그 다음에 [거기] 주변에 카페, 갔었어, 사람, 많더라 가 나타날 확률을 각각 높인다.
     - 즉, 스킵-그램 모델은 컨텍스트로 설정한 단어 주변에 어떤 단어들이 분포해 잇는지를 학습한다는 이야기.
